In [2]:
from pyserini.search import FaissSearcher, LuceneSearcher

from pyserini.search.faiss import AutoQueryEncoder
from pyserini.search import get_topics, get_qrels
import json
from tqdm import tqdm


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/xionggm/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.8/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/xionggm/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/xionggm/miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/xionggm/miniconda3 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/xionggm/miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/home/xionggm/.mujoco/mujoco210/bin')}
  warn(msg)


[2024-01-29 14:45:13,356] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
corpus = LuceneSearcher('index_relation_fb')

JavaException: JVM exception occurred: MemorySegmentIndexInputProvider is missing in Lucene JAR file java.lang.LinkageError

In [26]:
# BM25 search

In [27]:
bm25_searcher = LuceneSearcher('index_relation_fb')

In [28]:
hits = bm25_searcher.search('newton per metre is the unit of measurement for surface tension in which system of measurement?', k=20)
for hit in hits:
    print(json.loads(corpus.doc(str(hit.docid)).raw()))

{'id': '18202', 'contents': 'measurement unit surface tension unit tension in newtons per meter', 'rel_ori': 'measurement_unit.surface_tension_unit.tension_in_newtons_per_meter'}
{'id': '18203', 'contents': 'measurement unit surface tension unit measurement system', 'rel_ori': 'measurement_unit.surface_tension_unit.measurement_system'}
{'id': '20530', 'contents': 'measurement unit measurement system surface tension units', 'rel_ori': 'measurement_unit.measurement_system.surface_tension_units'}
{'id': '18525', 'contents': 'measurement unit unit of surface density measurement system', 'rel_ori': 'measurement_unit.unit_of_surface_density.measurement_system'}
{'id': '20509', 'contents': 'measurement unit measurement system surface density units', 'rel_ori': 'measurement_unit.measurement_system.surface_density_units'}
{'id': '17153', 'contents': 'measurement unit unit of volumetric flow rate rate in cubic metres per second', 'rel_ori': 'measurement_unit.unit_of_volumetric_flow_rate.rate_in_

In [29]:
# Contriever Search

In [34]:
query_encoder = AutoQueryEncoder(encoder_dir='facebook/contriever', pooling='mean')
contriever_searcher = FaissSearcher('freebase_contriever_index', query_encoder)

In [35]:
hits = contriever_searcher.search('newton per metre is the unit of measurement for surface tension in which system of measurement?', k=20)
for hit in hits:
    print(json.loads(corpus.doc(str(hit.docid)).raw()))

{'id': '18202', 'contents': 'measurement unit surface tension unit tension in newtons per meter', 'rel_ori': 'measurement_unit.surface_tension_unit.tension_in_newtons_per_meter'}
{'id': '20530', 'contents': 'measurement unit measurement system surface tension units', 'rel_ori': 'measurement_unit.measurement_system.surface_tension_units'}
{'id': '18525', 'contents': 'measurement unit unit of surface density measurement system', 'rel_ori': 'measurement_unit.unit_of_surface_density.measurement_system'}
{'id': '17162', 'contents': 'measurement unit unit of force force in newtons', 'rel_ori': 'measurement_unit.unit_of_force.force_in_newtons'}
{'id': '18526', 'contents': 'measurement unit unit of surface density density in kilograms per square meter', 'rel_ori': 'measurement_unit.unit_of_surface_density.density_in_kilograms_per_square_meter'}
{'id': '18203', 'contents': 'measurement unit surface tension unit measurement system', 'rel_ori': 'measurement_unit.surface_tension_unit.measurement_s

In [36]:
# hybrid searcher that combine BM25 and Contriever

In [37]:
hsearcher = HybridSearcher(contriever_searcher, bm25_searcher)
hits = hsearcher.search('newton per metre is the unit of measurement for surface tension in which system of measurement?', k=20)
for hit in hits:
    print(json.loads(corpus.doc(str(hit.docid)).raw()))

NameError: name 'HybridSearcher' is not defined